In [1]:
from utils import *

hete = Heterogeneity()
hete.check_torch_gpu()

-------------------------------------------------
------------------ VERSION INFO -----------------
Conda Environment: torchy | Python version: 3.8.16 (default, Mar  2 2023, 03:18:16) [MSC v.1916 64 bit (AMD64)]
Torch version: 2.0.1
Torch build with CUDA? True
# Device(s) available: 1, Name(s): Quadro P520



***
# END

In [13]:
n_realizations = 10
n_timesteps    = 61
dim            = 256

pressure, sat_h2 = np.zeros((n_realizations,n_timesteps,dim,dim,1)), np.zeros((n_realizations,n_timesteps,dim,dim,1))
poro,     perm   = np.zeros((n_realizations,dim,dim,1)),             np.zeros((n_realizations,dim,dim,1))

for i in range(n_realizations):
    data0 = loadmat('//dcstorage.lanl.gov/MFR2/misael/h2dataf/{}UHSS_0'.format(i+1))
    poro[i], perm[i] = data0['PORO'], data0['PERMX']
    for j in range(n_timesteps):
        data = loadmat('//dcstorage.lanl.gov/MFR2/misael/h2dataf/{}UHSS_{}'.format(i+1,j))
        pressure[i,j] = data['PRESSURE']
        sat_h2[i,j]   = data['SGAS'] * data['YMF_3']

In [14]:
facies = np.expand_dims(hete.load_facies()[:n_realizations],-1)
facies.shape

Facies shape: (1000, 256, 256)


(2, 256, 256, 1)

In [15]:
t_steps = np.arange(61)
times = np.ones((n_realizations,61,256,256,1))

for i in range(61):
    times[:,i] = times[:,i]*t_steps[i]

print(t_steps.shape, times.shape)

(61,) (2, 61, 256, 256, 1)


In [16]:
X_train = np.concatenate([np.repeat(np.concatenate([poro,perm,facies], -1)[:,np.newaxis,...], n_timesteps, axis=1),times],-1).reshape(n_realizations*n_timesteps,dim,dim,4)
y_train = np.concatenate([sat_h2, pressure],-1).reshape(n_realizations*n_timesteps,dim,dim,2)

X_train = np.moveaxis(X_train, -1, 1)
y_train = np.moveaxis(y_train, -1, 1)

print('X: {} | y: {}'.format(X_train.shape, y_train.shape))

X: (122, 4, 256, 256) | y: (122, 2, 256, 256)


In [34]:
device = 'cuda'

rom = h2_hete_rom().to(device)
optimizer = NAdam(rom.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

X_train = torch.Tensor(X_train).to(device)
y_train = torch.Tensor(y_train).to(device)

loss, val_loss = [], []
metrics = {'loss':[], 'val_loss':[]}
epochs = 10
batch_size = 50

for epoch in range(epochs):
    rom.train()
    epoch_loss = 0.0
    for i in range(0, len(X_train), batch_size):
        inp  = X_train[i:i+batch_size]
        true = y_train[i:i+batch_size]
        optimizer.zero_grad()
        pred = rom(inp)
        loss = loss_fn(pred,true)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()*inp.size(0)
    metrics['loss'].append(epoch_loss/len(X_train))

OutOfMemoryError: CUDA out of memory. Tried to allocate 200.00 MiB (GPU 0; 4.00 GiB total capacity; 2.61 GiB already allocated; 0 bytes free; 2.90 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF